### Vector Store를 활용한 RAG 구현

In [ ]:
import os, pprint
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv('../.env')

In [2]:
client = OpenAI(api_key=os.getenv("OPEN-AI"))

In [ ]:
file = client.files.create(
    file = open('../pdf/r-d-innovation.pdf', 'rb'),
    purpose = 'assistants'
)

vector_store = client.beta.vector_stores.create(
    name="R&D Innovation"
)

# 파일을 Vector Store에 추가
client.beta.vector_stores.files.create(
    vector_store_id=vector_store.id,
    file_id=file.id
)

In [4]:
# Assistant 생성
assistant = client.beta.assistants.create(
    name = "My Assistant",
    instructions = """
                넌 국가연구개발 혁신법에 대해 잘 아는 행정 관련 전문가야.
                만약 Q&A에 해당 내용이 있으면 이에 대해 자세하고 친절하게 답변해줘.
                정확하다고 생각되는 답을 찾았으면, 가장 먼저 "예" 혹은 "아니오"로 답변을 시작하고, 그렇지 않으면 "모르겠어"라고 말해줘.
                답한 내용의 근거를 정확하게 말해주되 문서내 검색한 내용만 답해 줘.
                마침표 다음에 개행 문자를 넣어줘.
    """,
    model="gpt-4-turbo-preview",
    tools=[
        {"type": "code_interpreter"},
        {"type": "file_search"}
    ],
    tool_resources={
        "file_search": {"vector_store_ids": [vector_store.id]}
    }
)

In [ ]:
# Thread 생성
thread = client.beta.threads.create()
print(f'thread id = {thread.id}')
# 메시지 추가
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="음식점에서 먹은 음식은 회의비로 처리가 가능해?"
)

# Run 생성 및 실행
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id
)

# Run 완료 대기
while run.status != "completed":
    run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)

# 결과 메시지 가져오기
messages = client.beta.threads.messages.list(thread_id=thread.id)
# 결과 출력
for message in messages:
    if message.role == "assistant":
        print(message.content[0].text.value)

In [ ]:
assistant.id